<a href="https://colab.research.google.com/github/wcliao1962/2025_DL/blob/master/Keras_Imdb_Introduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import os
import tarfile

下載IMDb，執行前請先建立資料夾data

In [ ]:
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="data/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

downloaded: ('data/aclImdb_v1.tar.gz', <http.client.HTTPMessage object at 0x7bb00140ff90>)


In [ ]:
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("data/aclImdb_v1.tar.gz", 'r:gz')
    result=tfile.extractall('data/')

# 1. Import Library

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

# 資料準備

In [ ]:
#讀取檔案

In [ ]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [ ]:
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list=[]

    positive_path=path + filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]

    negative_path=path + filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]

    print('read',filetype, 'files:',len(file_list))

    all_labels = ([1] * 12500 + [0] * 12500)

    all_texts  = []
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]

    return all_labels,all_texts

In [ ]:
y_train,train_text=read_files("train")

read train files: 25000


In [ ]:
y_test,test_text=read_files("test")

read test files: 25000


In [ ]:
#查看正面評價的影評

In [ ]:
train_text[0]

'Let us begin by saying that this film\'s English title "The Power of Kangwon Province" is an absolute misnomer.It is because in Hong Sang Soo\'s film,there are no actual shots of wars,troubles and conflicts.So the idea of establishing power of a province is neither suitable nor valid in the context of this film.If we were to judge this film by its Korean language title,"Kangwon-do ui him" is going to appear as a cryptic statement about emotional turmoils of its young protagonists whose minds are not at rest.Hong Sang Soo has also directed a highly prolific visual document about erratic choices made by people in their lives.The people in question are a couple of young girls who are constantly in the process of displaying their moods,whims and fancies. If making a film out of nothingness can be claimed as a film maker\'s meritorious virtue then Hong Sang Soo has to be saluted as a courageous film maker whose films speak volumes about ubiquitous nothingness of human relationships,sentime

In [ ]:
y_train[0]

1

In [ ]:
#查看負面評價的影評

In [ ]:
train_text[12499]

"Enjoyable in spite of Leslie Howard's performance. Mr. Howard plays Philip as a flat, uninteresting character. One is supposed to feel sorry for this man; however, I find myself cheering Bette Davis' Mildred. Ms. Davis gives one her finest performances (she received an Academy Award nomination). Thanks to her performance she brings this rather dull movie to life. **Be sure not to miss when Mildred tells Philip exactly how she feels about him."

In [ ]:
y_train[12499]

1

# 先讀取所有文章建立字典，限制字典的數量為nb_words=2000

In [ ]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [ ]:
#Tokenizer屬性

In [ ]:
#fit_on_texts 讀取多少文章

In [ ]:
print(token.document_count)

25000


In [ ]:
print(token.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'are': 22, 'his': 23, 'have': 24, 'be': 25, 'he': 26, 'one': 27, 'all': 28, 'at': 29, 'by': 30, 'an': 31, 'they': 32, 'who': 33, 'so': 34, 'from': 35, 'like': 36, 'her': 37, 'or': 38, 'just': 39, 'about': 40, "it's": 41, 'out': 42, 'has': 43, 'if': 44, 'some': 45, 'there': 46, 'what': 47, 'good': 48, 'more': 49, 'when': 50, 'very': 51, 'up': 52, 'no': 53, 'time': 54, 'she': 55, 'even': 56, 'my': 57, 'would': 58, 'which': 59, 'only': 60, 'story': 61, 'really': 62, 'see': 63, 'their': 64, 'had': 65, 'can': 66, 'were': 67, 'me': 68, 'well': 69, 'than': 70, 'we': 71, 'much': 72, 'been': 73, 'get': 74, 'bad': 75, 'will': 76, 'also': 77, 'do': 78, 'into': 79, 'people': 80, 'other': 81, 'first': 82, 'because': 83, 'great': 84, 'how': 85, 'him': 86, 'most': 87, "don't": 88, 'made

# 將每一篇文章的文字轉換一連串的數字
#只有在字典中的文字會轉換為數字

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [ ]:
print(train_text[0])

Let us begin by saying that this film's English title "The Power of Kangwon Province" is an absolute misnomer.It is because in Hong Sang Soo's film,there are no actual shots of wars,troubles and conflicts.So the idea of establishing power of a province is neither suitable nor valid in the context of this film.If we were to judge this film by its Korean language title,"Kangwon-do ui him" is going to appear as a cryptic statement about emotional turmoils of its young protagonists whose minds are not at rest.Hong Sang Soo has also directed a highly prolific visual document about erratic choices made by people in their lives.The people in question are a couple of young girls who are constantly in the process of displaying their moods,whims and fancies. If making a film out of nothingness can be claimed as a film maker's meritorious virtue then Hong Sang Soo has to be saluted as a courageous film maker whose films speak volumes about ubiquitous nothingness of human relationships,sentiments 

In [ ]:
print(x_train_seq[0])

[383, 174, 890, 30, 656, 11, 10, 591, 627, 421, 1, 666, 4, 6, 31, 1551, 8, 6, 83, 7, 18, 46, 22, 53, 774, 659, 4, 1643, 2, 34, 1, 322, 4, 666, 4, 3, 6, 1078, 881, 7, 1, 4, 10, 18, 44, 71, 67, 5, 1916, 10, 18, 30, 90, 1100, 421, 78, 86, 6, 166, 5, 972, 13, 3, 40, 915, 4, 90, 181, 619, 22, 20, 29, 356, 43, 77, 522, 3, 541, 1109, 40, 89, 30, 80, 7, 64, 452, 1, 80, 7, 883, 22, 3, 374, 4, 181, 535, 33, 22, 1338, 7, 1, 1767, 4, 64, 2, 44, 227, 3, 18, 42, 4, 66, 25, 13, 3, 18, 91, 43, 5, 25, 13, 3, 18, 619, 104, 1124, 40, 4, 402, 1514, 2, 452, 722, 27, 1226, 8, 38, 20, 10, 6, 1, 60, 1247, 1172, 11, 25, 35, 10, 839, 18]


# 讓轉換後的數字長度相同

In [ ]:
#
#文章內的文字，轉換為數字後，每一篇的文章地所產生的數字長度都不同，因為後需要進行類神經網路的訓練，所以每一篇文章所產生的數字長度必須相同
#以下列程式碼為例maxlen=100，所以每一篇文章轉換為數字都必須為100
#bj6eji3t03g/ 2k

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=100)

In [ ]:
#如果文章轉成數字大於0,pad_sequences處理後，會truncate前面的數字

In [ ]:
print('before pad_sequences length=',len(x_train_seq[0]))
print(x_train_seq[0])

before pad_sequences length= 150
[383, 174, 890, 30, 656, 11, 10, 591, 627, 421, 1, 666, 4, 6, 31, 1551, 8, 6, 83, 7, 18, 46, 22, 53, 774, 659, 4, 1643, 2, 34, 1, 322, 4, 666, 4, 3, 6, 1078, 881, 7, 1, 4, 10, 18, 44, 71, 67, 5, 1916, 10, 18, 30, 90, 1100, 421, 78, 86, 6, 166, 5, 972, 13, 3, 40, 915, 4, 90, 181, 619, 22, 20, 29, 356, 43, 77, 522, 3, 541, 1109, 40, 89, 30, 80, 7, 64, 452, 1, 80, 7, 883, 22, 3, 374, 4, 181, 535, 33, 22, 1338, 7, 1, 1767, 4, 64, 2, 44, 227, 3, 18, 42, 4, 66, 25, 13, 3, 18, 91, 43, 5, 25, 13, 3, 18, 619, 104, 1124, 40, 4, 402, 1514, 2, 452, 722, 27, 1226, 8, 38, 20, 10, 6, 1, 60, 1247, 1172, 11, 25, 35, 10, 839, 18]


In [ ]:
print('after pad_sequences length=',len(x_train[0]))
print(x_train[0])

after pad_sequences length= 100
[  18   30   90 1100  421   78   86    6  166    5  972   13    3   40
  915    4   90  181  619   22   20   29  356   43   77  522    3  541
 1109   40   89   30   80    7   64  452    1   80    7  883   22    3
  374    4  181  535   33   22 1338    7    1 1767    4   64    2   44
  227    3   18   42    4   66   25   13    3   18   91   43    5   25
   13    3   18  619  104 1124   40    4  402 1514    2  452  722   27
 1226    8   38   20   10    6    1   60 1247 1172   11   25   35   10
  839   18]


In [ ]:
#如果文章轉成數字不足100,pad_sequences處理後，前面會加上0

In [ ]:
print('before pad_sequences length=',len(x_train_seq[1]))
print(x_train_seq[1])

before pad_sequences length= 166
[9, 292, 10, 232, 310, 15, 360, 1392, 1, 1001, 108, 9, 88, 628, 36, 89, 14, 244, 98, 2, 1705, 24, 9, 419, 173, 98, 17, 1, 7, 1, 207, 464, 5, 36, 10, 34, 9, 515, 8, 3, 576, 142, 1258, 9, 118, 8, 12, 180, 48, 1862, 1259, 2, 13, 528, 17, 7, 3, 92, 32, 67, 112, 13, 64, 1350, 2, 32, 67, 108, 2, 20, 652, 528, 38, 10, 16, 95, 609, 1, 16, 513, 42, 15, 94, 14, 1, 739, 11, 290, 154, 8, 12, 5, 25, 121, 14, 31, 340, 133, 32, 22, 14, 1, 516, 32, 22, 77, 40, 1, 197, 11, 32, 118, 291, 10, 6, 575, 7, 8, 12, 69, 220, 1, 152, 11, 32, 184, 14, 1, 1147, 2, 1862, 1259, 67, 346, 19, 2, 14, 3, 244, 16, 8, 184, 176, 5, 1, 40, 64, 143, 1989, 19, 2, 121, 1, 264, 9, 198, 10, 27, 3, 787, 160, 9, 419, 8, 11, 72]


In [ ]:
print('after pad_sequences length=',len(x_train[1]))
print(x_train[1])

after pad_sequences length= 100
[  67  108    2   20  652  528   38   10   16   95  609    1   16  513
   42   15   94   14    1  739   11  290  154    8   12    5   25  121
   14   31  340  133   32   22   14    1  516   32   22   77   40    1
  197   11   32  118  291   10    6  575    7    8   12   69  220    1
  152   11   32  184   14    1 1147    2 1862 1259   67  346   19    2
   14    3  244   16    8  184  176    5    1   40   64  143 1989   19
    2  121    1  264    9  198   10   27    3  787  160    9  419    8
   11   72]


# 資料預處理

In [ ]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=100)